In [0]:
from pyspark.sql import SparkSession

In [0]:
# Inicializa SparkSession
spark = SparkSession.builder.getOrCreate()

In [0]:
# Valida se coluna não possui valores nulos
def validate_not_null(table_name: str, column_name: str):
    df = spark.table(table_name)
    if df.filter(f"{column_name} IS NULL").count() > 0:
        raise Exception(f"[SILVER] Coluna {column_name} contém NULL em {table_name}.")

# Valida se coluna é única
def validate_unique(table_name: str, column_name: str):
    df = spark.table(table_name)
    duplicates = df.groupBy(column_name).count().filter("count > 1").count()
    if duplicates > 0:
        raise Exception(f"[SILVER] {column_name} não é único em {table_name}.")

# Valida se valores estão dentro de uma condição
def validate_range(table_name: str, condition: str, description: str):
    df = spark.table(table_name)
    if df.filter(condition).count() > 0:
        raise Exception(f"[SILVER] Regra inválida ({description}) em {table_name}.")

# Valida chave estrangeira entre tabelas
def validate_fk(fact_table: str, dim_table: str, fact_col: str, dim_col: str):
    fact = spark.table(fact_table)
    dim = spark.table(dim_table)

    invalid = fact.join(dim, fact[fact_col] == dim[dim_col], "left_anti").count()

    if invalid > 0:
        raise Exception(f"[SILVER] FK inválida {fact_col} em {fact_table}.")

In [0]:
def run_validations():
    # DIM_CLUBE
    validate_not_null("project_data_football_silver.dim_clube", "clube_id")
    validate_unique("project_data_football_silver.dim_clube", "clube_id")

    # DIM_ATLETA
    validate_not_null("project_data_football_silver.dim_atleta", "atleta_id")
    validate_unique("project_data_football_silver.dim_atleta", "atleta_id")
    validate_not_null("project_data_football_silver.dim_atleta", "posicao_id")

    # DIM_RODADA
    validate_unique("project_data_football_silver.dim_rodada", "rodada_id")
    validate_range(
        "project_data_football_silver.dim_rodada",
        "numero_rodada < 1 OR numero_rodada > 38",
        "numero_rodada fora do intervalo 1-38"
    )

    # FATO_PARTIDA
    validate_fk(
        "project_data_football_silver.fato_partida",
        "project_data_football_silver.dim_clube",
        "clube_casa_id",
        "clube_id"
    )
    validate_range(
        "project_data_football_silver.fato_partida",
        "gols_casa < 0 OR gols_visitante < 0",
        "gols negativos"
    )

    # FATO_PONTUACAO
    validate_not_null("project_data_football_silver.fato_pontuacao", "atleta_id")
    validate_fk(
        "project_data_football_silver.fato_pontuacao",
        "project_data_football_silver.dim_atleta",
        "atleta_id",
        "atleta_id"
    )

    print("Validações SILVER concluídas com sucesso.")


if __name__ == "__main__":
    run_validations()